# Non-Negative Matrix Factorisation

- Matrix Factorization is a general technique used in collaborative filtering and other applications where a matrix is decomposed into the product of two lower-rank matrices.
- Stochastic Gradient Descent is an optimization algorithm commonly used to minimize the error in the factorization process.

- The key idea is to iteratively update the elements of the factorized matrices using the gradient of the error with respect to the elements.


## Algorithm Summary

Matrix factorization is a class of collaborative filtering algorithms used in recommender systems. Matrix factorization algorithms work by decomposing the user-item interaction matrix. 

1. **Load the data**
- data is provided in a dataframe where each row is a review

2. **Create a user-item matrix**
- convert dataframe into user-item matrix where each row is a user and each column is an item

3. **Create test and train set**
- hide $N$ ratings for each user in the training set and use them to test the performance of the model
- Typically, a certain percentage of ratings for each user are masked in the training set and used for testing the model's performance.

4. **Apply Non-negative Matrix Factorization (NMF)**

    1.  Decompose the user-item interaction matrix into two non-negative matrices: a user matrix and an item matrix.
    2. Minimize the reconstruction error between the original matrix and the product of the decomposed matrices using optimization techniques like gradient descent.

5. **Make predictions**
- For each user-item pair in the test set, predict the rating by reconstructing the original rating matrix using the decomposed user and item matrices.
- The predicted rating is obtained by taking the dot product of the corresponding user and item latent factor vectors.

6. **Evaluate the model**
- Calculate the predictive accuracy of the model using various evaluation metrics such as Root Mean Squared Error (RMSE), Mean Squared Error (MSE), and Mean Absolute Error (MAE).
- Additionally, assess the Top-N recommendation performance of the model using metrics like Normalized Discounted Cumulative Gain (NDCG) and Hit Rate.


## Manaul / From Fundamentals

In [28]:
%reset -f

# load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [31]:
# load data
# amz_data = pd.read_csv(r'C:\Users\e1002902\Documents\GitHub Repository\Masters-Dissertation\Code\Data\set2_data_modelling.csv')
amz_data = pd.read_csv("/Users/pavansingh/Library/CloudStorage/GoogleDrive-pavansingho23@gmail.com/My Drive/Portfolio/Masters-Dissertation/Code/Data/set4_data_modelling.csv", index_col=0)
display(amz_data.head())

# print details
print('Number of Rows: ', amz_data.shape[0])
print('Number of Columns: ', amz_data.shape[1])
print('Number of Unique Users: ', len(amz_data['reviewerID'].unique()))
print('Number of Unique Products: ', len(amz_data['asin'].unique()))
print('Fewest reviews by a reviewer:', amz_data.groupby('reviewerID')['asin'].count().min())
print('Most reviews by a reviewer:', amz_data.groupby('reviewerID')['asin'].count().max())
print("Fewest reviews per product:", amz_data.groupby('asin')['reviewerID'].count().min())
print("Most reviews per product:", amz_data.groupby('asin')['reviewerID'].count().max())

# Creating User Item Matrix =====================================================
# create user-item matrix
x = amz_data.pivot_table(index='reviewerID', columns='asin', values='overall')
x = x.fillna(0)
print("\n\n\nUser-Item Matrix")
display(x.head())
print('Shape: ', x.shape)

,reviewerID,reviewTime,asin,overall,reviewText,stemmed_words_revText,lemmatised_reviewText,filtered_tokens_revText,sentiments_afinn,sentiments_bing,sentiments_vader
76,AQ8OO59DJFJNZ,2018-01-05,0767834739,5.0,wonderful movie,wonder movi,wonderful movie,wonderful movie,4,1,0.5719
78,A244CRJ2QSVLZ4,2008-01-29,0767834739,5.0,resident evil is a great science fictionhorror...,resid evil great scienc fictionhorror hybrid p...,resident evil great science fictionhorror hybr...,resident evil great science fictionhorror hybr...,-12,-5,-0.9455
81,A1VCLTAGM5RLND,2005-07-23,0767834739,5.0,i this movie has people living and working und...,movi peopl live work underground place call hi...,movie people living working underground place ...,movie people living working underground place ...,-1,0,-0.1806
82,A119Q9NFGVOEJZ,2016-02-13,0767834739,5.0,every single video game based movie from the s...,everi singl video game base movi super mario b...,every single video game based movie super mari...,every single video game based movie super mari...,18,6,0.9846
83,A1RP6YCOS5VJ5I,2006-09-26,0767834739,5.0,i think that i like this movie more than the o...,think like movi origin origin still great real...,think like movie original original still great...,think like movie original original still great...,29,10,0.9951


Number of Rows:  83139
Number of Columns:  11
Number of Unique Users:  3668
Number of Unique Products:  3249
Fewest reviews by a reviewer: 13
Most reviews by a reviewer: 193
Fewest reviews per product: 13
Most reviews per product: 189



User-Item Matrix


asin,0767834739,7799146915,B00000DMAT,B00000DMAX,B00000DMB3,B00000F1GM,B00000I1BJ,B00000I1BY,B00000ID61,B00000INR2,...,B01H353FLA,B01H353HUY,B01H3VFR6U,B01H5GB8ZW,B01H6OXQFS,B01H9SH2LU,B01HGBAFNC,B01HHVVLGQ,B01HHVWWMI,B01HIZF7XE
reviewerID,,,,,,,,,,,,,,,,,,,,,
A100RH4M1W1DF0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A100WO06OQR8BQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
A1027EV8A9PV1O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A103KKI1Y4TFNQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1047P9FLHTDZJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Shape:  (3668, 3249)


### Train and Test Split

In [36]:
# create a copy of the original matrix to store hidden ratings
x_hidden = x.copy()
indices_tracker = []

# number of products to hide for each user
N = 4

# identifies rated items and randomly selects N products to hide ratings for each user
np.random.seed(2207)  # You can use any integer value as the seed
for user_id in range(x_hidden.shape[0]):
    rated_products = np.where(x_hidden.iloc[user_id, :] > 0)[0]
    # print("User:", user_id)
    # print("Indices of Rated Products:", rated_products)
    hidden_indices = np.random.choice(rated_products, N, replace=False)
    indices_tracker.append(hidden_indices)
    # print("Indices to Hide:", hidden_indices, "\n")
    x_hidden.iloc[user_id, hidden_indices] = 0

In [37]:
# check tracker - all hidden ratings 
indices_tracker = pd.DataFrame(indices_tracker).to_numpy()
print("Indices of Ratings per user \n", indices_tracker)

# flattened
indices_tracker_flat = indices_tracker.flatten()
print("Indices of Ratings per User joined", indices_tracker_flat)

# see updated matrix with hidden ratings
print("\n\nUpdated Matrix with Hidden Ratings")
display(x_hidden)

# see original matrix
print("Original Matrix")
display(x)

Indices of Ratings per user 
 [[2807 2258 2647 1953]
 [2111 1398 1498 1710]
 [ 200 1102 1089 1429]
 ...
 [2353 1482  185 1469]
 [ 639 2206 3123  311]
 [ 193  533  406  220]]
Indices of Ratings per User joined [2807 2258 2647 ...  533  406  220]


Updated Matrix with Hidden Ratings


asin,0767834739,7799146915,B00000DMAT,B00000DMAX,B00000DMB3,B00000F1GM,B00000I1BJ,B00000I1BY,B00000ID61,B00000INR2,...,B01H353FLA,B01H353HUY,B01H3VFR6U,B01H5GB8ZW,B01H6OXQFS,B01H9SH2LU,B01HGBAFNC,B01HHVVLGQ,B01HHVWWMI,B01HIZF7XE
reviewerID,,,,,,,,,,,,,,,,,,,,,
A100RH4M1W1DF0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A100WO06OQR8BQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
A1027EV8A9PV1O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A103KKI1Y4TFNQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1047P9FLHTDZJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZVIQ5SU7XPD5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZW0HVDKOXGN9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZX2RDN9YXZAE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Original Matrix


asin,0767834739,7799146915,B00000DMAT,B00000DMAX,B00000DMB3,B00000F1GM,B00000I1BJ,B00000I1BY,B00000ID61,B00000INR2,...,B01H353FLA,B01H353HUY,B01H3VFR6U,B01H5GB8ZW,B01H6OXQFS,B01H9SH2LU,B01HGBAFNC,B01HHVVLGQ,B01HHVWWMI,B01HIZF7XE
reviewerID,,,,,,,,,,,,,,,,,,,,,
A100RH4M1W1DF0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A100WO06OQR8BQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
A1027EV8A9PV1O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A103KKI1Y4TFNQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1047P9FLHTDZJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZVIQ5SU7XPD5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZW0HVDKOXGN9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZX2RDN9YXZAE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Decomposition, Optimisation and Prediction

The implementation of matrix factorization using stochastic gradient descent (SGD) for non-negative matrix factorization (NMF). Let’s break down the key components:

1. Initialization:
- We initialize matrices P and Q with non-negative random values, which is appropriate for NMF.
- The bias terms b_u and b_i are initialized to zeros, and the global bias b is calculated as the mean of non-zero elements in the input matrix R.



2. Update Rules:
- We use the SGD approach to update P and Q iteratively based on the error eij (difference between the actual rating and the predicted rating).
- If use_regularization is enabled, We apply L2 regularization to the updates by adding a penalty term proportional to the current value of P and Q.
- The bias terms b_u and b_i are also updated based on the error.
- Main Loop: The code runs for a fixed number of iterations, with each iteration looping over all $(i,j)$, entries in the input matrix $R$. If R[i][j] is a non-zero rating, it computes the prediction error $e_{ij}$ (difference between observed and predicted ratings). Then, it updates P[i] and Q[j] by SGD with added regularization terms.

3. Convergence Check:
- We monitor the convergence by calculating the Frobenius norm of the difference between the original matrix R and the reconstructed matrix PQ^T.
- If the difference falls below a threshold (0.001 in our case), the algorithm stops iterating.

4. Bias Terms:
- We correctly add the bias terms to the final prediction if use_bias is enabled.

In [62]:
def matrix_factorization_sgd(R, K, steps=50, alpha=0.001, beta=0.02, use_regularization=True, use_bias=True):
    # R = user-item ratings matrix
    # K = number of latent features
    # steps = number of iterations
    # alpha = learning rate
    # beta = bias term

    N, M = R.shape
    P = np.abs(np.random.randn(N, K))  # Initialize with non-negative values
    Q = np.abs(np.random.randn(M, K))
    counter = 0

    # Initialize bias terms
    if use_bias:
        b_u = np.zeros(N)
        b_i = np.zeros(M)
        b = np.mean(R[np.where(R != 0)])  # global bias

    for step in range(steps):
        print("Step:", step)
        for i in range(N):
            for j in range(M):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i, :], Q[j, :])

                    # Update P and Q
                    for k in range(K):
                        if use_regularization:
                            P[i][k] += alpha * (2 * eij * Q[j][k] - beta * P[i][k])
                            Q[j][k] += alpha * (2 * eij * P[i][k] - beta * Q[j][k])
                        else:
                            P[i][k] += alpha * (2 * eij * Q[j][k])
                            Q[j][k] += alpha * (2 * eij * P[i][k])

                    # Update bias terms
                    if use_bias:
                        b_u[i] += alpha * (eij - beta * b_u[i])
                        b_i[j] += alpha * (eij - beta * b_i[j])

        # Check for convergence within the loop
        if np.sqrt(np.sum((R - np.dot(P, Q.T))**2)) < 0.001:
            break

    # Add bias terms to the prediction
    if use_bias:
        R_pred = np.dot(P, Q.T) + b + b_u[:, np.newaxis] + b_i[np.newaxis:,]  
    else:
        R_pred = np.dot(P, Q.T)

    return P, Q, R_pred


# Use the function to reconstruct the original matrix
np.random.seed(42)
R = x_hidden.values
nP, nQ, nR_pred = matrix_factorization_sgd(R, K=2, alpha=0.001, beta=0.02, use_regularization=True, use_bias=True)
print("Original Matrix:")
print(R)
print("\nReconstructed Matrix:")
print(nR_pred)

#  convert the reconstructed matrix to a dataframe
nR_pred = pd.DataFrame(nR_pred, columns=x_hidden.columns, index=x_hidden.index)
print("\nReconstructed Matrix as a DataFrame")
display(nR_pred)

Step: 0
Step: 1
Step: 2
Step: 3
Step: 4
Step: 5
Step: 6
Step: 7
Step: 8
Step: 9
Step: 10
Step: 11
Step: 12
Step: 13
Step: 14
Step: 15
Step: 16
Step: 17
Step: 18
Step: 19
Step: 20
Step: 21
Step: 22
Step: 23
Step: 24
Step: 25
Step: 26
Step: 27
Step: 28
Step: 29
Step: 30
Step: 31
Step: 32
Step: 33
Step: 34
Step: 35
Step: 36
Step: 37
Step: 38
Step: 39
Step: 40
Step: 41
Step: 42
Step: 43
Step: 44
Step: 45
Step: 46
Step: 47
Step: 48
Step: 49
Original Matrix:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

Reconstructed Matrix:
[[ 8.11170986 10.40847241 10.91051086 ... 10.0319263  10.42355291
  10.2637877 ]
 [ 7.86299632  9.63079303  9.99850803 ...  9.38918653  9.50124324
   9.67077695]
 [ 8.09068375 10.40319009 10.90813577 ... 10.02503282 10.41706243
  10.25923443]
 ...
 [ 5.30186922  8.25856039  8.98029402 ...  7.5987452   8.71749544
   7.5860218 ]
 [ 6.85017054  9.7408542  10.406726

asin,0767834739,7799146915,B00000DMAT,B00000DMAX,B00000DMB3,B00000F1GM,B00000I1BJ,B00000I1BY,B00000ID61,B00000INR2,...,B01H353FLA,B01H353HUY,B01H3VFR6U,B01H5GB8ZW,B01H6OXQFS,B01H9SH2LU,B01HGBAFNC,B01HHVVLGQ,B01HHVWWMI,B01HIZF7XE
reviewerID,,,,,,,,,,,,,,,,,,,,,
A100RH4M1W1DF0,8.111710,10.408472,10.910511,10.048930,10.798227,10.493628,10.508752,9.997847,10.148841,10.406584,...,9.613887,9.788183,10.115304,10.604661,10.019414,9.791733,9.771988,10.031926,10.423553,10.263788
A100WO06OQR8BQ,7.862996,9.630793,9.998508,9.220483,9.987230,9.726718,9.733856,9.052886,9.402366,9.397979,...,8.998871,9.176625,9.140778,9.995920,9.457581,8.757086,8.804417,9.389187,9.501243,9.670777
A1027EV8A9PV1O,8.090684,10.403190,10.908136,10.041008,10.796725,10.491079,10.506611,9.987088,10.142911,10.400852,...,9.602777,9.778602,10.103728,10.605033,10.012206,9.782307,9.759400,10.025033,10.417062,10.259234
A103KKI1Y4TFNQ,7.192071,9.708995,10.279979,9.420201,10.079278,9.753159,9.769571,9.492981,9.433007,9.890751,...,8.858497,9.011851,9.643021,9.707174,9.176330,9.340167,9.306429,9.247536,9.850172,9.413032
A1047P9FLHTDZJ,7.915208,10.250739,10.762679,9.895135,10.642789,10.334944,10.350645,9.852923,9.988805,10.266159,...,9.444283,9.618240,9.972635,10.433985,9.845625,9.653452,9.628851,9.864177,10.276709,10.092193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZVIQ5SU7XPD5,7.593515,9.963377,10.490180,9.634454,10.341876,10.029676,10.044935,9.635327,9.697830,10.035040,...,9.151921,9.316461,9.766515,10.075472,9.518911,9.449204,9.426814,9.554506,10.029793,9.757770
AZW0HVDKOXGN9,6.197117,8.371460,8.880402,8.144746,8.656363,8.352730,8.360072,8.286306,8.109236,8.571175,...,7.648873,7.767143,8.456006,8.234668,7.875684,8.105976,8.142328,7.942372,8.545087,8.052676
AZX2RDN9YXZAE,5.301869,8.258560,8.980294,8.160691,8.556620,8.184346,8.201294,8.556232,7.946534,8.894651,...,7.308165,7.400191,8.791628,7.733555,7.385289,8.525853,8.478086,7.598745,8.717495,7.586022


### Grid Search for Tuning

In [54]:
hidden_ratings_ind = indices_tracker.copy()
hidden_ratings_arrays = []
for user in range(x.shape[0]):
    user_hidden_ratings = x.iloc[user, hidden_ratings_ind[user, :]].reset_index(drop=True).values
    hidden_ratings_arrays.append(user_hidden_ratings)

hidden_ratings_array = pd.DataFrame(hidden_ratings_arrays).to_numpy().flatten()

In [64]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import itertools

# Define the hyperparameters to tune
param_grid = {
    'K': [2,3,5],         # Number of latent features
    'alpha': [0.001, 0.0001], # Learning rate
    'beta': [0.1, 0.5, 1]    # Regularization parameter
}

# Create all possible combinations of hyperparameters
param_combinations = list(itertools.product(*param_grid.values()))

# Initialize variables to keep track of the best parameters and the best RMSE
best_params = None
best_rmse = float('inf')  # initialize with a large value
counter = 0

# Loop over each parameter combination
for params in param_combinations:
    
    # Unpack the parameters
    K, alpha, beta = params
    
    # counter
    counter += 1

    # Run matrix factorization with the current hyperparameters
    np.random.seed(42)
    print(f"Iteration {counter} of {len(param_combinations)}")
    print(f'K={K}, alpha={alpha}, beta={beta}')
    nP, nQ, nR_pred = matrix_factorization_sgd(
        R, K=K, alpha=alpha, beta=beta, use_regularization=True, use_bias=True)
    
    # Compute RMSE
    nR_pred = pd.DataFrame(nR_pred, columns=x_hidden.columns, index=x_hidden.index)
    predicted_ratings_arrays = []
    for user in range(nR_pred.shape[0]):
        user_predicted_ratings = nR_pred.iloc[user, hidden_ratings_ind[user, :]].reset_index(drop=True).values
        predicted_ratings_arrays.append(user_predicted_ratings)

    predicted_ratings_array = pd.DataFrame(predicted_ratings_arrays).to_numpy().flatten()
    rmse = np.sqrt(mean_squared_error(hidden_ratings_array, predicted_ratings_array))

    # Check if this is the best RMSE so far
    print(f"Checking RMSE: {rmse}")
    if rmse < best_rmse:
        print(f'New best RMSE: {rmse}')
        best_rmse = rmse
        best_params = params
    else :
        print("RMSE not improved")
    print("\n")

# Print the best parameters and the best RMSE
print(f'Best Parameters: {best_params}')
print(f'Best RMSE: {best_rmse}')


Iteration 1 of 18
K=2, alpha=0.001, beta=0.1
Step: 0
Step: 1
Step: 2
Step: 3
Step: 4
Step: 5
Step: 6
Step: 7
Step: 8
Step: 9
Step: 10
Step: 11
Step: 12
Step: 13
Step: 14
Step: 15
Step: 16
Step: 17
Step: 18
Step: 19
Step: 20
Step: 21
Step: 22
Step: 23
Step: 24
Step: 25
Step: 26
Step: 27
Step: 28
Step: 29
Step: 30
Step: 31
Step: 32
Step: 33
Step: 34
Step: 35
Step: 36
Step: 37
Step: 38
Step: 39
Step: 40
Step: 41
Step: 42
Step: 43
Step: 44
Step: 45
Step: 46
Step: 47
Step: 48
Step: 49
Checking RMSE: 5.136051412482668
New best RMSE: 5.136051412482668


Iteration 2 of 18
K=2, alpha=0.001, beta=0.5
Step: 0
Step: 1
Step: 2
Step: 3
Step: 4
Step: 5
Step: 6
Step: 7
Step: 8
Step: 9
Step: 10
Step: 11
Step: 12
Step: 13
Step: 14
Step: 15
Step: 16
Step: 17
Step: 18
Step: 19
Step: 20
Step: 21
Step: 22
Step: 23
Step: 24
Step: 25
Step: 26
Step: 27
Step: 28
Step: 29
Step: 30
Step: 31
Step: 32
Step: 33
Step: 34
Step: 35
Step: 36
Step: 37
Step: 38
Step: 39
Step: 40
Step: 41
Step: 42
Step: 43
Step: 44
Step: 4

KeyboardInterrupt: 

### Evaluation (Predictive Accuracy)

Now evaluate how good the predictions are vs the hidden ratings
- ***step 1***: identify the hidden ratings indices
- ***step 2***: extract hidden ratings indices and corresponding predicted ratings indices
- ***step 3***: calculate MAE, MSE and RMSE (take the hidden ratings as the true values and the predicted ratings as the predicted values

In [44]:
# step 1: identify the hidden ratings indices = indices_tracker and get the hidden ratings ==========================================================================
hidden_ratings_ind = indices_tracker.copy()

# Loop through users to append hidden ratings
hidden_ratings_arrays = []

# Loop through users to append hidden ratings arrays
for user in range(x.shape[0]):
    user_hidden_ratings = x.iloc[user, hidden_ratings_ind[user, :]].reset_index(drop=True).values
    hidden_ratings_arrays.append(user_hidden_ratings)


hidden_ratings_array = pd.DataFrame(hidden_ratings_arrays).to_numpy().flatten()
print("Hidden Ratings:", hidden_ratings_array)

# step 2: extract corresponding predicted ratings indices ==========================================================================

# Create an empty list to store predicted ratings arrays
predicted_ratings_arrays = []

# Loop through users to append predicted ratings arrays
for user in range(nR_pred.shape[0]):
    user_predicted_ratings = nR_pred.iloc[user, hidden_ratings_ind[user, :]].reset_index(drop=True).values
    predicted_ratings_arrays.append(user_predicted_ratings)

predicted_ratings_array = pd.DataFrame(predicted_ratings_arrays).to_numpy().flatten()
print("Corresponding Predicted Ratings:", predicted_ratings_array)

# step 3: calculate MAE, MSE and RMSE (take the hidden ratings as the true values and the predicted ratings as the predicted values) ==========================================================================

from sklearn.metrics import mean_absolute_error, mean_squared_error

# calculate MAE, MSE and RMSE
print("Using sklearn")
mae = mean_absolute_error(hidden_ratings_array, predicted_ratings_array)
mse = mean_squared_error(hidden_ratings_array, predicted_ratings_array)
rmse = np.sqrt(mse)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")


# Manually
print("\n\nManually")
mae = np.mean(np.abs(hidden_ratings_array - predicted_ratings_array)) # Calculate Mean Absolute Error (MAE)
mse = np.mean((hidden_ratings_array - predicted_ratings_array) ** 2) # Calculate Mean Squared Error (MSE)
rmse = np.sqrt(mse) # Calculate Root Mean Squared Error (RMSE)


print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

Hidden Ratings: [3. 5. 5. ... 4. 1. 5.]
Corresponding Predicted Ratings: [5.7297655  5.69016655 6.06080439 ... 4.81111752 5.00394805 5.15794999]
Using sklearn
Mean Absolute Error (MAE): 1.8010954363377594
Mean Squared Error (MSE): 5.308916394493818
Root Mean Squared Error (RMSE): 2.3041085899961002


Manually
Mean Absolute Error (MAE): 1.8010954363377594
Mean Squared Error (MSE): 5.308916394493818
Root Mean Squared Error (RMSE): 2.3041085899961002


In [ ]:
# round to 2 decimal places
mae = round(mae, 3)
mse = round(mse, 3)
rmse = round(rmse, 3)

# Save the results to a csv file
results = pd.DataFrame({'MAE': [mae], 'MSE': [mse], 'RMSE': [rmse]})
# results.to_csv(r'C:\Users\e1002902\Documents\GitHub Repository\Masters-Dissertation\Code\Data\results_NMF.csv', index=False)
results.to_csv('/Users/pavansingh/Library/CloudStorage/GoogleDrive-pavansingho23@gmail.com/My Drive/Portfolio/Masters-Dissertation/Code/Data/Results/MF_results.csv', index=False)

## Using Packages

- The `use_regularization` parameter controls whether regularization (beta) is applied.
- The `use_bias` parameter controls whether bias terms are included.
- We use the `NMF` class from `Scikit-learn`, which handles the optimization process for us.

In [26]:
# import NMF from scikit-learn
from sklearn.decomposition import NMF


def matrix_factorization_nmf(R, K, steps=500, alpha=0.001, beta=0.02, use_regularization=True, use_bias=True):
    """
    Perform non-negative matrix factorization using scikit-learn's NMF.

    Args:
        R (numpy.ndarray): The input rating matrix.
        K (int): Number of latent features.
        steps (int): Maximum number of iterations.
        alpha (float): Learning rate.
        beta (float): Regularization parameter.
        use_regularization (bool): Whether to use regularization.
        use_bias (bool): Whether to use bias terms.
    Returns:
        numpy.ndarray, numpy.ndarray, numpy.ndarray: Factorized matrices P, Q, and the reconstructed matrix R_pred.
    """

    # Initialize NMF model
    nmf_model = NMF(n_components=K, init='random', solver='cd', beta_loss='frobenius', max_iter=steps,
                    alpha=alpha if use_regularization else 0.0, l1_ratio=beta if use_regularization else 0)

    # Fit the model to the data
    nmf_model.fit(R)

    # Get the transformed matrices
    P = nmf_model.transform(R)
    Q = nmf_model.components_

    if use_bias:
        b_u = np.zeros(P.shape[0])
        b_i = np.zeros(Q.shape[1])
        b = np.mean(R[np.where(R != 0)])

        for _ in range(steps):
            for i in range(P.shape[0]):
                for j in range(Q.shape[1]):
                    if R[i][j] > 0:
                        eij = R[i][j] - np.dot(P[i, :], Q[:, j])

                        P[i, :] += alpha * (2 * eij * Q[:, j] - beta * P[i, :])
                        Q[:, j] += alpha * (2 * eij * P[i, :] - beta * Q[:, j])

                        b_u[i] += alpha * (eij - beta * b_u[i])
                        b_i[j] += alpha * (eij - beta * b_i[j])

        R_pred = np.dot(P, Q) + b + b_u[:, np.newaxis] + b_i[np.newaxis:,]
    else:
        R_pred = np.dot(P, Q)

    return P, Q, R_pred

# Example usage
np.random.seed(42)
nP, nQ, nR_pred = matrix_factorization_nmf(R, K=2, alpha=0.001, beta=0.02, use_regularization=True, use_bias=True)
print("Original Matrix:")
print(R)
print("\nReconstructed Matrix:")
print(nR_pred)

/Users/pavansingh/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1477: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  warnings.warn(
/Users/pavansingh/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1477: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  warnings.warn(


Original Matrix:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

Reconstructed Matrix:
[[ 9.47914175  4.52876106  4.70464802 ...  4.52876106  4.52876106
   4.88160201]
 [ 9.47914175  4.52876106  4.70464802 ...  4.52876106  4.52876106
   4.88160201]
 [ 9.47914175  4.52876106  4.70464802 ...  4.52876106  4.52876106
   4.88160201]
 ...
 [11.96670806  7.01632738  7.19221434 ...  7.01632738  7.01632738
   7.36916833]
 [ 9.47914175  4.52876106  4.70464802 ...  4.52876106  4.52876106
   4.88160201]
 [ 9.47914175  4.52876106  4.70464802 ...  4.52876106  4.52876106
   4.88160201]]


In [27]:
# Define the parameter grid
param_grid = {
    'K': [2, 3, 4],         # Number of latent features
    'alpha': [0.001, 0.01], # Learning rate
    'beta': [0.01, 0.02]    # Regularization parameter
}

# Create an instance of the GridSearchCV
grid_search = GridSearchCV(estimator=matrix_factorization_nmf, param_grid=param_grid, cv=5)

# Perform grid search
grid_search.fit(R)

# get params
best_K = grid_search.best_params_['K']
best_beta = grid_search.best_params_['beta']
best_alpha = grid_search.best_params_['alpha']


# Print the best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)
print(f"Best K: {best_K}")
print(f"Best beta: {best_beta}")
print(f"Best alpha: {best_alpha}")

# Re-train the model with best hyperparameters
best_model = matrix_factorization_nmf(R=x_hidden.values, K=best_K, alpha=best_beta, beta=best_beta,
                                      use_regularization=True, use_bias=True)

TypeError: estimator should be an estimator implementing 'fit' method, <function matrix_factorization_nmf at 0x7faa32abc550> was passed

In [ ]:
# step 1: identify the hidden ratings indices = indices_tracker and get the hidden ratings ==========================================================================
hidden_ratings_ind = indices_tracker.copy()

# Loop through users to append hidden ratings
hidden_ratings_arrays = []

# Loop through users to append hidden ratings arrays
for user in range(x.shape[0]):
    user_hidden_ratings = x.iloc[user, hidden_ratings_ind[user, :]].reset_index(drop=True).values
    hidden_ratings_arrays.append(user_hidden_ratings)


hidden_ratings_array = pd.DataFrame(hidden_ratings_arrays).to_numpy().flatten()
print("Hidden Ratings:", hidden_ratings_array)

# step 2: extract corresponding predicted ratings indices ==========================================================================

# Create an empty list to store predicted ratings arrays
predicted_ratings_arrays = []

# Loop through users to append predicted ratings arrays
for user in range(nR.shape[0]):
    user_predicted_ratings = nR.iloc[user, hidden_ratings_ind[user, :]].reset_index(drop=True).values
    predicted_ratings_arrays.append(user_predicted_ratings)

predicted_ratings_array = pd.DataFrame(predicted_ratings_arrays).to_numpy().flatten()
print("Corresponding Predicted Ratings:", predicted_ratings_array)

# step 3: calculate MAE, MSE and RMSE (take the hidden ratings as the true values and the predicted ratings as the predicted values) ==========================================================================

from sklearn.metrics import mean_absolute_error, mean_squared_error

# calculate MAE, MSE and RMSE
print("Using sklearn")
mae = mean_absolute_error(hidden_ratings_array, predicted_ratings_array)
mse = mean_squared_error(hidden_ratings_array, predicted_ratings_array)
rmse = np.sqrt(mse)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")


# Manually
print("\n\nManually")
mae = np.mean(np.abs(hidden_ratings_array - predicted_ratings_array)) # Calculate Mean Absolute Error (MAE)
mse = np.mean((hidden_ratings_array - predicted_ratings_array) ** 2) # Calculate Mean Squared Error (MSE)
rmse = np.sqrt(mse) # Calculate Root Mean Squared Error (RMSE)


print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

## Using Packages without Bias and Regularisation

In [84]:
import numpy as np
from sklearn.decomposition import NMF

# Create a copy of the original ratings matrix (you should use a copy so as not to modify the original matrix)
R_copy = R.copy()

# Replace original zeros in R_copy with NaNs
R_copy[R_copy == 0] = np.nan

# Specify the number of components (you can experiment with different values)
n_components = 10
model = NMF(n_components=n_components, init='random', random_state=2207, max_iter=1000, alpha=0.01, l1_ratio=1.5, verbose=False)
P = model.fit_transform(R_copy)  # User-feature matrix
Q = model.components_            # Feature-item matrix

# Multiply P and Q to get the estimated ratings
R_estimated = np.dot(P, Q)

# Create a mask for the missing values
mask = np.isnan(R_copy)

# Replace the original missing values with the predicted ratings
R_predicted = R.copy()  # Create a copy to ensure that the original matrix is not modified
R_predicted[mask] = R_estimated[mask]

# Print the original and predicted ratings
print("Original Ratings:")
print(R)

print("\nPredicted Ratings:")
print(R_predicted)

ValueError: Input X contains NaN.
NMF does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [82]:
import numpy as np
from sklearn.decomposition import NMF

# Specify the number of components (you can experiment with different values)
n_components = 10
model = NMF(n_components=n_components, init='random', random_state=2207, max_iter=1000, alpha=0.01, l1_ratio=1.5, verbose=False)
P = model.fit_transform(R)  # User-feature matrix
Q = model.components_       # Feature-item matrix


# Multiply A and B to get the estimated ratings
R_estimated = np.dot(P, Q)

# Replace original zeros in R with predicted ratings
R_predicted = np.where(R == 0, R_estimated, R)

# Print the original and predicted ratings
print("Original Ratings:")
print(R)

print("\nPredicted Ratings:")
print(R_predicted)
R_predicted = pd.DataFrame(R_predicted, columns=x_hidden.columns, index=x_hidden.index)

/Users/pavansingh/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:1477: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  warnings.warn(


Original Ratings:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

Predicted Ratings:
[[1.13756107e-05 1.97293066e-04 0.00000000e+00 ... 1.75846414e-01
  1.80200509e-01 3.25668829e-03]
 [5.13774224e-03 1.35576910e-04 1.39125748e-04 ... 1.80595700e-02
  1.84806670e-02 8.05029405e-03]
 [3.44309594e-03 3.36838948e-03 1.52714421e-02 ... 1.62260490e-03
  0.00000000e+00 5.91030722e-03]
 ...
 [3.90118221e-07 2.51222987e-04 6.20551495e-04 ... 1.46281673e-04
  1.16376849e-05 1.55002994e-03]
 [0.00000000e+00 2.96481398e-04 0.00000000e+00 ... 7.20876672e-02
  7.38726141e-02 1.73037274e-03]
 [6.46961783e-02 2.17507721e-02 1.33368959e-01 ... 7.24586383e-04
  2.20241253e-04 1.88555567e-03]]


In [83]:
# step 1: identify the hidden ratings indices = indices_tracker and get the hidden ratings ==========================================================================
hidden_ratings_ind = indices_tracker.copy()

# Loop through users to append hidden ratings
hidden_ratings_arrays = []

# Loop through users to append hidden ratings arrays
for user in range(x.shape[0]):
    user_hidden_ratings = x.iloc[user, hidden_ratings_ind[user, :]].reset_index(drop=True).values
    hidden_ratings_arrays.append(user_hidden_ratings)
hidden_ratings_array = pd.DataFrame(hidden_ratings_arrays).to_numpy().flatten()

# step 2: extract corresponding predicted ratings indices ==========================================================================

# Create an empty list to store predicted ratings arrays
predicted_ratings_arrays = []

# Loop through users to append predicted ratings arrays
for user in range(R_predicted.shape[0]):
    user_predicted_ratings = R_predicted.iloc[user, hidden_ratings_ind[user, :]].reset_index(drop=True).values
    predicted_ratings_arrays.append(user_predicted_ratings)
predicted_ratings_array = pd.DataFrame(predicted_ratings_arrays).to_numpy().flatten()

# step 3: calculate MAE, MSE and RMSE (take the hidden ratings as the true values and the predicted ratings as the predicted values) ==========================================================================

from sklearn.metrics import mean_absolute_error, mean_squared_error

# calculate MAE, MSE and RMSE
print("Using sklearn")
mae = mean_absolute_error(hidden_ratings_array, predicted_ratings_array)
mse = mean_squared_error(hidden_ratings_array, predicted_ratings_array)
rmse = np.sqrt(mse)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")


# Manually
print("\n\nManually")
mae = np.mean(np.abs(hidden_ratings_array - predicted_ratings_array)) # Calculate Mean Absolute Error (MAE)
mse = np.mean((hidden_ratings_array - predicted_ratings_array) ** 2) # Calculate Mean Squared Error (MSE)
rmse = np.sqrt(mse) # Calculate Root Mean Squared Error (RMSE)


print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

Using sklearn
Mean Absolute Error (MAE): 4.241243101668249
Mean Squared Error (MSE): 18.985869875308374
Root Mean Squared Error (RMSE): 4.35727780561538


Manually
Mean Absolute Error (MAE): 4.241243101668249
Mean Squared Error (MSE): 18.985869875308374
Root Mean Squared Error (RMSE): 4.35727780561538


## Using Suprise Test and Train Split

In [ ]:
# load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# load data
# amz_data = pd.read_csv(r'C:\Users\e1002902\Documents\GitHub Repository\Masters-Dissertation\Code\Data\set2_data_modelling.csv')
amz_data = pd.read_csv("/Users/pavansingh/Library/CloudStorage/GoogleDrive-pavansingho23@gmail.com/My Drive/Portfolio/Masters-Dissertation/Code/Data/set4_data_modelling.csv", index_col=0)

# print details
print('Number of Rows: ', amz_data.shape[0])
print('Number of Columns: ', amz_data.shape[1])
print('Number of Unique Users: ', len(amz_data['reviewerID'].unique()))
print('Number of Unique Products: ', len(amz_data['asin'].unique()))
print('Fewest reviews by a reviewer:', amz_data.groupby('reviewerID')['asin'].count().min())
print('Most reviews by a reviewer:', amz_data.groupby('reviewerID')['asin'].count().max())
print("Fewest reviews per product:", amz_data.groupby('asin')['reviewerID'].count().min())
print("Most reviews per product:", amz_data.groupby('asin')['reviewerID'].count().max())

# Creating User Item Matrix =====================================================
# create user-item matrix
x = amz_data.pivot_table(index='reviewerID', columns='asin', values='overall')
x = x.fillna(0)
print("\n\n\nUser-Item Matrix")
display(x.head())
print('Shape: ', x.shape)

,reviewerID,reviewTime,asin,overall,reviewText,stemmed_words_revText,lemmatised_reviewText,filtered_tokens_revText,sentiments_afinn,sentiments_bing,sentiments_vader
76,AQ8OO59DJFJNZ,2018-01-05,0767834739,5.0,wonderful movie,wonder movi,wonderful movie,wonderful movie,4,1,0.5719
78,A244CRJ2QSVLZ4,2008-01-29,0767834739,5.0,resident evil is a great science fictionhorror...,resid evil great scienc fictionhorror hybrid p...,resident evil great science fictionhorror hybr...,resident evil great science fictionhorror hybr...,-12,-5,-0.9455
81,A1VCLTAGM5RLND,2005-07-23,0767834739,5.0,i this movie has people living and working und...,movi peopl live work underground place call hi...,movie people living working underground place ...,movie people living working underground place ...,-1,0,-0.1806
82,A119Q9NFGVOEJZ,2016-02-13,0767834739,5.0,every single video game based movie from the s...,everi singl video game base movi super mario b...,every single video game based movie super mari...,every single video game based movie super mari...,18,6,0.9846
83,A1RP6YCOS5VJ5I,2006-09-26,0767834739,5.0,i think that i like this movie more than the o...,think like movi origin origin still great real...,think like movie original original still great...,think like movie original original still great...,29,10,0.9951


Number of Rows:  83139
Number of Columns:  11
Number of Unique Users:  3668
Number of Unique Products:  3249
Fewest reviews by a reviewer: 13
Most reviews by a reviewer: 193
Fewest reviews per product: 13
Most reviews per product: 189



User-Item Matrix


asin,0767834739,7799146915,B00000DMAT,B00000DMAX,B00000DMB3,B00000F1GM,B00000I1BJ,B00000I1BY,B00000ID61,B00000INR2,...,B01H353FLA,B01H353HUY,B01H3VFR6U,B01H5GB8ZW,B01H6OXQFS,B01H9SH2LU,B01HGBAFNC,B01HHVVLGQ,B01HHVWWMI,B01HIZF7XE
reviewerID,,,,,,,,,,,,,,,,,,,,,
A100RH4M1W1DF0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A100WO06OQR8BQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
A1027EV8A9PV1O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A103KKI1Y4TFNQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1047P9FLHTDZJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Shape:  (3668, 3249)


### Train and Test Split

In [ ]:
# load hidden ratings matrix
x_hidden = pd.read_csv('/Users/pavansingh/Library/CloudStorage/GoogleDrive-pavansingho23@gmail.com/My Drive/Portfolio/Masters-Dissertation/Code/Data/suprise_hidden_ratings_matrix.csv')

# load testset indices
testset_indices = pd.read_csv('/Users/pavansingh/Library/CloudStorage/GoogleDrive-pavansingho23@gmail.com/My Drive/Portfolio/Masters-Dissertation/Code/Data/suprise_testset_indices.csv')

# convert to numpy
testset_indices = testset_indices.to_numpy()

In [ ]:
#  get predicted ratings for the testset
predicted_ratings = []
for i in range(len(testset_indices)):
    user_id = testset_indices[i][0]
    item_id = testset_indices[i][1]
    predicted_ratings.append(predic_matrix.iloc[user_id, item_id])

print("Predicted Ratings:")
print(predicted_ratings)

# get actual ratings for the testset
print("\nActual Ratings:")
actual_ratings = testset_df[2].to_list()
print(actual_ratings)

In [ ]:
# calculate MAE, MSE and RMSE
from sklearn.metrics import mean_absolute_error, mean_squared_error
print("Using sklearn")
mae = mean_absolute_error(actual_ratings, predicted_ratings)
mse = mean_squared_error(actual_ratings, predicted_ratings)
rmse = np.sqrt(mse)

print(f"Mean Absolute Error (MAE): {mae.round(2)}")
print(f"Mean Squared Error (MSE): {mse.round(2)}")
print(f"Root Mean Squared Error (RMSE): {rmse.round(2)}")


# Manually
print("\n\nManually")

# calculate MAE, MSE and RMSE using actual and predicted ratings
mae = np.mean(np.abs(np.array(actual_ratings) - np.array(predicted_ratings))) # Calculate Mean Absolute Error (MAE)
mse = np.mean((np.array(actual_ratings) - np.array(predicted_ratings)) ** 2) # Calculate Mean Squared Error (MSE)
rmse = np.sqrt(mse) # Calculate Root Mean Squared Error (RMSE)

print(f"Mean Absolute Error (MAE): {mae.round(2)}")
print(f"Mean Squared Error (MSE): {mse.round(2)}")
print(f"Root Mean Squared Error (RMSE): {rmse.round(2)}")

In [ ]:
# save results to csv
results = pd.DataFrame({'MAE': [mae.round(3)], 'MSE': [mse.round(3)], 'RMSE': [rmse.round(3)]})
results.to_csv("Data/Results/NMF_results.csv", index=False)
